# Approximate School Name Matching

Matches school names and their ID's

In [74]:
import pandas as pd
from fuzzywuzzy import fuzz, process

In [75]:
school_to_id = pd.read_excel('GA 2017-2018 school names and IDs.xlsx')
educator_inex = pd.read_csv('Educators_Inexperienced_2018_JAN_24th_2019.csv')
educator_out = pd.read_csv('Educators_OUT_OF_FIELD_2018_JAN_24th_2019.csv')

In [76]:
process.extractOne('Altamaha Elementary School', school_to_id['SCHOOL_NAME'], scorer=fuzz.partial_ratio)

('Altamaha Elementary School', 100, 4)

In [77]:
def match_school_to_id(school_name, school_to_id):
    result = process.extractOne(school_name, school_to_id['SCHOOL_NAME'], scorer=fuzz.partial_ratio)
    matching_name = result[0]
    confience = result[1]
    row_index = result[2]
    school_id = school_to_id.iloc[row_index]['Full School_ID']
    return school_id, matching_name, confience

In [78]:
educator_out_res = educator_out['INSTN_NAME'].apply(lambda school_name: match_school_to_id(school_name, school_to_id))

In [79]:
educator_out['SCHOOL_ID'] = educator_out_res.apply(lambda x: x[0])
educator_out['MATCHING_NAME'] = educator_out_res.apply(lambda x: x[1])
educator_out['CONFIDENCE'] = educator_out_res.apply(lambda x: x[2])

In [85]:
educator_out.to_csv('Educators_OUT_OF_FIELD_2018_JAN_24th_2019_with_id.csv', index=False)

In [81]:
educator_inex_res = educator_inex['INSTN_NAME'].apply(lambda school_name: match_school_to_id(school_name, school_to_id))

In [82]:
educator_inex['SCHOOL_ID'] = educator_inex_res.apply(lambda x: x[0])
educator_inex['MATCHING_NAME'] = educator_inex_res.apply(lambda x: x[1])
educator_inex['CONFIDENCE'] = educator_inex_res.apply(lambda x: x[2])

In [84]:
educator_inex.to_csv('Educators_Inexperienced_2018_JAN_24th_2019_with_id.csv', index=False)